# Text-to-3D Task

## 1. Load the model

In [68]:
import torch
import requests
import numpy as np
from io import BytesIO
from diffusers import DiffusionPipeline, ShapEPipeline
from PIL import Image
import os
import time
import trimesh
from diffusers.utils import export_to_gif



In [ ]:
# "bhuvanmdev/LGM-f"

# there is some bug with the GLM model
# pipeline = DiffusionPipeline.from_pretrained(
#     "bhuvanmdev/LGM-f",
#     custom_pipeline="bhuvanmdev/LGM-f",
#     torch_dtype=torch.float16,
#     trust_remote_code=True,
# ).to("cuda")

pipeline = ShapEPipeline.from_pretrained(
    "openai/shap-e",
    torch_dtype=torch.float16,
    trust_remote_code=True,
).to("cuda")




model_index.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

c:\Users\28262\anaconda3\envs\3dgen\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\28262\.cache\huggingface\hub\models--openai--shap-e. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

prior%2Fconfig.json:   0%|          | 0.00/489 [00:00<?, ?B/s]

text_encoder%2Fconfig.json:   0%|          | 0.00/626 [00:00<?, ?B/s]

...

tokenizer%2Fvocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

tokenizer%2Ftokenizer_config.json:   0%|          | 0.00/737 [00:00<?, ?B/s]

The config attributes {'renderer': ['shap_e', 'ShapERenderer']} were passed to ShapEPipeline, but are not expected and will be ignored. Please verify your model_index.json configuration file.
Keyword arguments {'renderer': ['shap_e', 'ShapERenderer'], 'trust_remote_code': True} are not expected by ShapEPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

An error occurred while trying to fetch C:\Users\28262\.cache\huggingface\hub\models--openai--shap-e\snapshots\7bd337afdea1c17842e1c3cc45c4e268356dba40\prior: Error no file named diffusion_pytorch_model.safetensors found in directory C:\Users\28262\.cache\huggingface\hub\models--openai--shap-e\snapshots\7bd337afdea1c17842e1c3cc45c4e268356dba40\prior.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch C:\Users\28262\.cache\huggingface\hub\models--openai--shap-e\snapshots\7bd337afdea1c17842e1c3cc45c4e268356dba40\shap_e_renderer: Error no file named diffusion_pytorch_model.safetensors found in directory C:\Users\28262\.cache\huggingface\hub\models--openai--shap-e\snapshots\7bd337afdea1c17842e1c3cc45c4e268356dba40\shap_e_renderer.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


In [ ]:
input_prompt = "a cup"

result = pipeline(
    input_prompt,
    num_inference_steps=50,
    guidance_scale=10,
    output_type="mesh",
).images




  0%|          | 0/99 [00:00<?, ?it/s]

2


In [ ]:
input_prompt = "a cup"
result = pipeline(
    input_prompt,
    num_inference_steps=100,
    guidance_scale=10,
)
# print(result.keys())
# print(result[0].shape, result[1].shape)
gaussains = result[0]

result_dir = os.path.join(os.getcwd(), "output")
os.makedirs(result_dir, exist_ok=True)
result_path = os.path.join(result_dir, f"test_{int(time.time())}_{input_prompt.replace(' ', '_')}.ply")

# pipeline.save_ply(gaussains, result_path)
pipeline.save_mesh(gaussains, result_path)

#ply to obj
mesh = trimesh.load(result_path)
mesh.export(os.path.join(result_dir, f"test_{int(time.time())}_{input_prompt.replace(' ', '_')}.obj"))



  0%|          | 0/199 [00:00<?, ?it/s]

AttributeError: 'ShapEPipeline' object has no attribute 'save_ply'

## 2. Build a gradio app

In [60]:
import gradio as gr

In [61]:
with gr.Blocks() as demo:
    gr.Model3D(result_path)

demo.launch()


* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.
